In [21]:
import pandas as pd
import numpy as np
import pandas_ta as ta 

In [25]:
df = pd.read_csv("btc.csv", index_col=0)
df = df.rename(columns={"open":"Open", "high":"High","low":"Low", "close":"Close","volume":"Volume"})
df.index = pd.to_datetime(df.index, format='%d-%m-%Y %H:%M')
df["sma_20"] = ta.sma(df["Close"],20)
df["sma_40"] = ta.sma(df["Close"],40)
df["sma_100"] = ta.sma(df["Close"],100)
df

,Open,High,Low,Close,Volume,sma_20,sma_40,sma_100
date,,,,,,,,
2022-09-01 00:00:00,20041.4,20108.6,19999.0,20103.6,4584.505,NaN,NaN,NaN
2022-09-01 00:05:00,20103.5,20126.2,20080.1,20125.7,2088.559,NaN,NaN,NaN
2022-09-01 00:10:00,20125.7,20125.8,20088.2,20099.4,1480.762,NaN,NaN,NaN
2022-09-01 00:15:00,20099.5,20122.5,20071.5,20071.7,1490.185,NaN,NaN,NaN
2022-09-01 00:20:00,20071.7,20086.5,20055.6,20057.2,1245.659,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2023-10-31 23:40:00,34629.9,34640.0,34620.3,34633.0,348.818,34579.220,34584.7875,34485.879
2023-10-31 23:45:00,34633.0,34650.0,34610.5,34648.2,638.422,34586.525,34585.7025,34489.642
2023-10-31 23:50:00,34648.2,34673.1,34645.4,34670.4,786.992,34594.080,34586.9450,34492.772


In [26]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

class SMAStrategy(Strategy):
    def init(self):
        
        self.take_profit_multiplier = 2.0
        self.stop_loss_multiplier = 1.0
        super().init()
        self.sma20 = self.I(SMA, self.data.Close, 20)
        self.sma40 = self.I(SMA, self.data.Close, 40)
        self.sma100 = self.I(SMA, self.data.Close, 100)

    def next(self):
        price = self.data.Close[-1]

        # Long Entry Condition
        if self.sma20[-1] > self.sma100[-1]:
                self.buy(sl=price * (1 - self.stop_loss_multiplier), 
                         tp=price * (1 + self.take_profit_multiplier), size=0.1)

        # Short Entry Condition
        if self.sma20[-1] < self.sma100[-1]:
                self.sell(sl=price * (1 + self.stop_loss_multiplier), 
                          tp=price * (1 - self.take_profit_multiplier), size=0.1)

        # Long Exit Condition
        for trade in self.trades:
            if trade.is_long and trade.is_open:
                if self.sma20[-1] < self.sma100[-1]:
                    trade.close() 

        
        for trade in self.trades:
            if trade.is_short and trade.is_open:
                if self.sma20[-1] > self.sma100[-1]:
                    trade.close()

# Backtesting the strategy with hedging enabled
bt = Backtest(df, SMAStrategy, cash=10000, commission=.002, trade_on_close=True, hedging=True)
stats = bt.run()

C:\Users\saumi\AppData\Local\Temp\ipykernel_17208\3924569177.py:46: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SMAStrategy, cash=10000, commission=.002, trade_on_close=True, hedging=True)
C:\Users\saumi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\backtesting\_plotting.py:122: UserWarning: Data contains too many candlesticks to plot; downsampling to '2H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


Row(id='1235', ...)

In [28]:
stats

Start                     2022-09-01 00:00:00
End                       2023-10-31 23:55:00
Duration                    425 days 23:55:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]               77.338387
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

In [29]:
stats["_trades"]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
